In [693]:
import pandas as pd
from sklearn import preprocessing
from category_encoders import TargetEncoder
import numpy as np
from sklearn import metrics
import lightgbm as lgb
import scipy.stats as stats
import xgboost as xgb

In [694]:
train = pd.read_csv('../input/train.csv')

In [695]:
train['Loan Status'].value_counts()

0    61222
1     6241
Name: Loan Status, dtype: int64

In [696]:
test = pd.read_csv('../input/test.csv')

In [697]:
pd.options.display.max_columns = 4000

In [698]:
test['Loan Status'] = -1

In [699]:
test.shape

(28913, 35)

In [700]:
full_data = pd.concat([train, test])

In [701]:
full_data.drop(['Payment Plan', 'Accounts Delinquent'],inplace = True,axis=1)

lt = pd.DataFrame(full_data['Loan Title'].value_counts())
lt.sort_index(inplace = True)
lt.to_csv('lt.csv',index=True)

In [702]:
cc = ['CC','CC Consolidation','CC Loan','CC Refi','CC Refinance ','CC consolidation ','CC-Refinance','Card Consolidation','Cards','Credit','Credit Card','Credit Card Consolidation','Credit Card Debt','Credit Card Loan','Credit Card Paydown','Credit Card Payoff','Credit Card Refi','Credit Card Refinance','Credit Card Refinance Loan','Credit Card consolidation','Credit Cards','Credit card pay off','Credit card payoff','Credit card refinance','Credit card refinancing','cards','credit card','credit card consolidation','credit card refinance']
refinance = ['CC Refi','CC Refinance','CC-Refinance','Credit Card Refi','Credit Card Refinance','Credit Card Refinance Loan','Credit card refinance','Credit card refinancing','Refinance','Refinance Loan','credit card refinance','refi']
consolidation = ['Bill Consolidation','CONSOLIDATE','CONSOLIDATION','Consolidate','Consolidated','Consolidation','Consolidation Loan','DEBT CONSOLIDATION','Debt','Debt Consolidation','Debt Consolidation 2013','Debt Consolidation Loan','Debt consolidation','Dept consolidation','consolidate','consolidation','consolidation loan','debt consolidation','debt consolidation loan']
home = ['Home','Home Improvement','Home Improvement Loan','Home buying','Home improvement','Home loan','home improvement','Bathroom']
car = ['Car Loan','Car financing']


In [703]:
full_data['credit card'] = np.where(full_data['Loan Title'].isin(cc),1,0)
full_data['refinance'] = np.where(full_data['Loan Title'].isin(refinance),1,0)
full_data['consolidation'] = np.where(full_data['Loan Title'].isin(consolidation),1,0)
full_data['home loan'] = np.where(full_data['Loan Title'].isin(home),1,0)
full_data['car loan'] = np.where(full_data['Loan Title'].isin(car),1,0)

In [704]:
def transformations(train,col,tr):
    if tr == 'log':
        train[col] = np.log(train[col])
        #val[col] = np.log(val[col])
    elif tr == 'boxcox':
        train[col] = stats.boxcox(train[col])
        #val[col] = stats.boxcox(val[col])

    elif tr == 'sqrt':
        train[col] = np.sqrt(train[col])
        #val[col] = np.sqrt(val[col])
    return train

In [705]:
transformations(full_data,'Home Ownership','log')
transformations(full_data,'Open Account','log')
transformations(full_data,'Total Accounts','sqrt')
transformations(full_data,'Total Received Interest','log')
transformations(full_data,'Total Received Late Fee','log')
transformations(full_data,'Recoveries','log')
transformations(full_data,'Collection Recovery Fee','log')
#transformations(full_data,'Total Collection Amount','boxcox')
transformations(full_data,'Total Current Balance','log')
transformations(full_data,'Total Revolving Credit Limit','log')

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,Home Ownership,Verification Status,Loan Title,Debit to Income,Delinquency - two years,Inquires - six months,Open Account,Public Record,Revolving Balance,Revolving Utilities,Total Accounts,Initial List Status,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status,credit card,refinance,consolidation,home loan,car loan
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,12.080207,Not Verified,Debt Consolidation,16.284758,1,0,2.564949,0,24246,74.932551,2.645751,w,7.982637,-2.282241,0.915607,-0.231020,0,INDIVIDUAL,49,31,12.648516,8.797700,0,0,0,1,0,0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,10.592474,Source Verified,Debt consolidation,15.412409,0,0,2.484907,0,812,78.297186,3.605551,f,6.649981,-3.319216,0.865930,-0.025501,0,INDIVIDUAL,109,53,12.115108,9.946786,0,0,0,1,0,0
2,1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,11.424167,Source Verified,Debt Consolidation,28.137619,0,0,2.639057,0,1843,2.073040,4.472136,w,6.760791,2.932721,1.462393,0.019876,0,INDIVIDUAL,66,34,11.405351,10.171796,0,0,0,1,0,0
3,6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,11.592536,Source Verified,Debt consolidation,18.043730,1,0,1.945910,0,13819,67.467951,3.464102,w,5.663562,-3.120584,-2.234737,-0.287721,0,INDIVIDUAL,39,40,9.125762,11.005660,0,0,0,1,0,0
4,14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,10.697268,Source Verified,Credit card refinancing,17.209886,1,3,2.564949,1,1544,85.250761,4.690416,w,4.861668,2.960449,7.166126,-0.997086,0,INDIVIDUAL,18,430,11.744267,10.024776,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28908,11537800,35000,35000,35000.00000,60,BAT5629144,16.240000,C,C5,MORTGAGE,12.793859,Verified,Credit card refinancing,7.310000,0,0,2.079442,1,83274,72.200000,4.000000,f,9.221906,-inf,-inf,-inf,0,INDIVIDUAL,109,0,13.692194,11.643954,-1,1,1,0,0,0
28909,11538258,25000,25000,25000.00000,60,BAT5629144,14.980000,C,C3,MORTGAGE,11.561716,Source Verified,Debt consolidation,10.220000,0,1,2.302585,0,26356,50.900000,5.099020,f,8.778626,-inf,-inf,-inf,0,INDIVIDUAL,104,0,13.149833,10.855145,-1,0,0,1,0,0
28910,13147685,30000,30000,29950.00000,60,BAT2136391,18.240000,D,D5,MORTGAGE,11.213171,Verified,Debt consolidation,27.300000,0,0,2.944439,0,31303,47.700000,6.708204,f,8.959978,-inf,-inf,-inf,1,INDIVIDUAL,83,0,12.599107,11.091331,-1,0,0,1,0,0
28911,13917689,25000,25000,24950.00000,60,BAT2136391,15.610000,D,D1,MORTGAGE,11.775290,Verified,Debt consolidation,27.290000,2,0,2.484907,0,25470,83.800000,5.830952,f,8.614959,-inf,-inf,-inf,0,INDIVIDUAL,83,0,12.968566,10.322198,-1,0,0,1,0,0


In [706]:
full_data.drop(['Loan Title'],inplace = True,axis=1)

In [707]:
full_data.shape

(96376, 37)

In [708]:
ohecols = ['Term','Grade','Employment Duration','Verification Status','Initial List Status','Application Type',]
tecols = ['Batch Enrolled','Sub Grade']


In [709]:
encoder = TargetEncoder()
# encoder.fit(train['Batch Enrolled'],train['Loan Status'])

In [710]:
for c in tecols:
    encoder = TargetEncoder()
    encoder.fit(train[c], train['Loan Status'])
    full_data[c] = encoder.transform(full_data[c])
    

In [711]:
full_data.shape

(96376, 37)

In [712]:
full_data = pd.get_dummies(full_data,columns=ohecols,drop_first=True)

In [713]:
traindf = full_data.loc[full_data['Loan Status'] != -1,:]
testdf = full_data.loc[full_data['Loan Status'] == -1,:]
testdf.drop(['Loan Status'],inplace=True,axis=1)

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [640]:
from sklearn import model_selection

X = traindf.drop('Loan Status',axis=1)
y = traindf.pop('Loan Status')

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.5, random_state=42)

In [716]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)

In [717]:
for fold, (train_idx, val_idx) in enumerate(kf.split(X=X, y=y)):
    traindf.loc[val_idx, 'kfold'] = fold

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Vasuki.K\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [643]:
traindf['kfold'].value_counts()

0.0    13493
1.0    13493
2.0    13493
3.0    13492
4.0    13492
Name: kfold, dtype: int64

def objective(trial,data = traindf.drop('Loan Status',axis=1),target=traindf['Loan Status']):
    train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    logloss = metrics.log_loss(valid_y, pred_labels)
    return logloss


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

#print("  Value: {}".format(trial.value))
#print("  Params: ")
#for key, value in trial.params.items():
#    print(key,value)
params = trial.params
params

for i in FOLD_MAPPPING:
    train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(i))].reset_index(drop=True)
    valid_df = traindf[traindf.kfold==i].reset_index(drop=True)

    ytrain = train_df['Loan Status'].values
    yvalid = valid_df['Loan Status'].values

    train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
    valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)


In [644]:
def trainfunc(traindf,trainresult):
    cvmodels = []
    FOLD_MAPPING = {
        0: [1, 2, 3, 4],
        1: [0, 2, 3, 4],
        2: [0, 1, 3, 4],
        3: [0, 1, 2, 4],
        4: [0, 1, 2, 3]
    }
    
    models = {#'rf' : ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=2),
              'xg' :  XGBClassifier(silent=1),
              'lgbm' : lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)}
    for i in FOLD_MAPPING:
        train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(i))].reset_index(drop=True)
        valid_df = traindf[traindf.kfold==i].reset_index(drop=True)

        ytrain = train_df['Loan Status'].values
        yvalid = valid_df['Loan Status'].values
        
        #tid = train_df['ID'].values
        vid = valid_df['ID'].values
        train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
        valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)
        
        valid_df = valid_df[train_df.columns]
        

        
        for k,v in models.items():
            cvmodels.append(v.fit(train_df,ytrain))
    return cvmodels
            #preds = v.predict_proba(valid_df)[:, 1]
            #trainresult[['id'+ str(i),'prob'+ str(i)]] = vid, preds   
            #cvmodels[str(i)].append(v.fit(train_df,ytrain))
            #colname = 'pred_' + str(k) +'_'+ str(i)
            #trainresult[colname] =  v.predict_proba(valid_df)[:, 1]
     

In [645]:
mods = trainfunc(traindf,trainresult)

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: Th

In [646]:
def valresult(traindf):
    FOLD_MAPPING = {
            0: [1, 2, 3, 4],
            1: [0, 2, 3, 4],
            2: [0, 1, 3, 4],
            3: [0, 1, 2, 4],
            4: [0, 1, 2, 3]
            }
    valres = dict()
    fold = 0
    logloss = 0
    for i,j in enumerate(mods):
        print(i)
        if (i%2 == 0) & (i!=0):
            valres[i/2] = logloss/2
            fold+=1
            logloss = 0
        valid_df = traindf[traindf.kfold==fold].reset_index(drop=True)
        yvalid = valid_df['Loan Status'].values
        valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)
        preds = j.predict_proba(valid_df)[:, 1]
        logloss += metrics.log_loss(yvalid, preds)
    return valres

In [647]:
v = valresult(traindf)
v

0
1
2
3
4
5
6
7
8
9


{1.0: 0.23589761086429814,
 2.0: 0.23525452473836464,
 3.0: 0.237443516086634,
 4.0: 0.23536256746112744}

In [648]:
def testfunc(mods,df):
    sub = pd.DataFrame()
    tid = df['ID']
    df = df.drop('ID',axis=1)
    for i,j in enumerate(mods):
        sub['preds'+ str(i)] = j.predict_proba(df)[:,-1]
    sub['final'] = sub.mean(axis = 1)
    return sub

In [649]:
res = testfunc(mods,testdf)

In [663]:
res = pd.DataFrame()

In [522]:
res['Loan Status'] = np.where(res['final']>=0.5,1,0)

In [523]:
res = res['Loan Status']

In [525]:
res.to_csv("submission.csv", index=False)

In [560]:
testdf

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Batch Enrolled,Interest Rate,Sub Grade,Home Ownership,Debit to Income,Delinquency - two years,Inquires - six months,Open Account,Public Record,Revolving Balance,Revolving Utilities,Total Accounts,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Last week Pay,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,credit card,refinance,consolidation,home loan,car loan,Term_58,Term_59,Term_60,Grade_B,Grade_C,Grade_D,Grade_E,Grade_F,Grade_G,Employment Duration_OWN,Employment Duration_RENT,Verification Status_Source Verified,Verification Status_Verified,Initial List Status_w,Application Type_JOINT
0,56492997,17120,10365,16025.082690,0.094373,12.163926,0.093523,11.244638,16.749219,1,0,2.484907,1,3576,67.278287,2.236068,8.405021,-2.430063,2.131296,-0.312252,0,135,24,13.072000,8.381144,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
1,22540813,7133,11650,12615.795600,0.086559,6.564296,0.089326,10.547419,18.157975,0,0,2.397895,0,1932,71.313157,4.582576,6.901644,-3.188420,1.817591,-0.007107,0,56,1,11.190127,7.852828,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0
2,9862181,25291,25825,11621.280830,0.104027,14.729900,0.094201,10.844977,15.190011,0,0,3.091042,0,598,50.883065,4.795832,6.591829,-3.828362,1.741356,-1.267339,0,3,26,12.559630,9.887155,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,10097822,30781,9664,15375.823510,0.087944,10.523767,0.097986,11.139947,21.292550,0,0,2.397895,0,5222,82.449083,5.291503,6.573495,-2.381646,0.904092,-0.041695,0,21,32,10.617466,8.885441,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0
4,47771809,8878,9419,7176.647582,0.096085,9.997013,0.082521,11.424715,4.812117,0,0,2.397895,0,553,49.075855,3.000000,5.515736,-4.570371,0.755105,-0.910519,0,104,33,11.416690,10.171413,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28908,11537800,35000,35000,35000.000000,0.098231,16.240000,0.095874,12.793859,7.310000,0,0,2.079442,1,83274,72.200000,4.000000,9.221906,-inf,-inf,-inf,0,109,0,13.692194,11.643954,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
28909,11538258,25000,25000,25000.000000,0.098231,14.980000,0.094201,11.561716,10.220000,0,1,2.302585,0,26356,50.900000,5.099020,8.778626,-inf,-inf,-inf,0,104,0,13.149833,10.855145,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
28910,13147685,30000,30000,29950.000000,0.092179,18.240000,0.103484,11.213171,27.300000,0,0,2.944439,0,31303,47.700000,6.708204,8.959978,-inf,-inf,-inf,1,83,0,12.599107,11.091331,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
28911,13917689,25000,25000,24950.000000,0.092179,15.610000,0.093523,11.775290,27.290000,2,0,2.484907,0,25470,83.800000,5.830952,8.614959,-inf,-inf,-inf,0,83,0,12.968566,10.322198,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0


In [718]:
testdf = testdf.drop('ID',axis=1)

In [719]:
xg = XGBClassifier()

In [720]:
train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(0))].reset_index(drop=True)
valid_df = traindf[traindf.kfold==0].reset_index(drop=True)

ytrain = train_df['Loan Status'].values
yvalid = valid_df['Loan Status'].values

#tid = train_df['ID'].values
vid = valid_df['ID'].values
train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)

valid_df = valid_df[train_df.columns]
xg.fit(train_df, ytrain)
preds2 = xg.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds2))
lgbm.fit(train_df, ytrain)
preds3 = lgbm.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds3))

res['xg1'] = xg.predict_proba(testdf)[:, 1]
res['lgb1'] = lgbm.predict_proba(testdf)[:, 1]

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.3241767533555437
0.3113653300550827


In [721]:
train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(1))].reset_index(drop=True)
valid_df = traindf[traindf.kfold==1].reset_index(drop=True)

ytrain = train_df['Loan Status'].values
yvalid = valid_df['Loan Status'].values

#tid = train_df['ID'].values
vid = valid_df['ID'].values
train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)

valid_df = valid_df[train_df.columns]
xg.fit(train_df, ytrain)
preds2 = xg.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds2))
lgbm.fit(train_df, ytrain)
preds3 = lgbm.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds3))

res['xg2'] = xg.predict_proba(testdf)[:, 1]
res['lgb2'] = lgbm.predict_proba(testdf)[:, 1]

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.32403844639951307
0.3098305903929026


In [722]:
train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(2))].reset_index(drop=True)
valid_df = traindf[traindf.kfold==2].reset_index(drop=True)

ytrain = train_df['Loan Status'].values
yvalid = valid_df['Loan Status'].values

#tid = train_df['ID'].values
vid = valid_df['ID'].values
train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)

valid_df = valid_df[train_df.columns]
xg.fit(train_df, ytrain)
preds2 = xg.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds2))
lgbm.fit(train_df, ytrain)
preds3 = lgbm.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds3))

res['xg3'] = xg.predict_proba(testdf)[:, 1]
res['lgb3'] = lgbm.predict_proba(testdf)[:, 1]

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.3226433758179254
0.3093363338115058


In [723]:
train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(3))].reset_index(drop=True)
valid_df = traindf[traindf.kfold==3].reset_index(drop=True)

ytrain = train_df['Loan Status'].values
yvalid = valid_df['Loan Status'].values

#tid = train_df['ID'].values
vid = valid_df['ID'].values
train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)

valid_df = valid_df[train_df.columns]
xg.fit(train_df, ytrain)
preds2 = xg.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds2))
lgbm.fit(train_df, ytrain)
preds3 = lgbm.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds3))

res['xg4'] = xg.predict_proba(testdf)[:, 1]
res['lgb4'] = lgbm.predict_proba(testdf)[:, 1]

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.3233214886621431
0.3105238289194345


In [724]:
train_df = traindf[traindf.kfold.isin(FOLD_MAPPPING.get(4))].reset_index(drop=True)
valid_df = traindf[traindf.kfold==4].reset_index(drop=True)

ytrain = train_df['Loan Status'].values
yvalid = valid_df['Loan Status'].values

#tid = train_df['ID'].values
vid = valid_df['ID'].values
train_df = train_df.drop(["ID", "Loan Status", "kfold"], axis=1)
valid_df = valid_df.drop(["ID", "Loan Status", "kfold"], axis=1)

valid_df = valid_df[train_df.columns]
xg.fit(train_df, ytrain)
preds2 = xg.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds2))
lgbm.fit(train_df, ytrain)
preds3 = lgbm.predict_proba(valid_df)[:, 1]
print(metrics.log_loss(yvalid, preds3))

res['xg5'] = xg.predict_proba(testdf)[:, 1]
res['lgb5'] = lgbm.predict_proba(testdf)[:, 1]

C:\Users\Vasuki.K\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.3223340793080924
0.3104629349377021


In [725]:
res['xg'] = (res['xg1'] + res['xg2'] + res['xg3'] + res['xg4'] + res['xg5'])/5
res['lgb'] = (res['lgb1'] + res['lgb2'] + res['lgb3'] + res['lgb4'] + res['lgb5'])/5

In [728]:
res[res['xg']>=0.4].shape

(93, 12)

In [729]:
res['Loan Status'] = np.where(res['xg']>0.35,1,0)

In [731]:
res.to_csv('submission.csv',index=None)